# Bayesian Networks in Python

In this tutorial, we will explore how to use Bayesian networks in Python using the `pgmpy` library.

See documentation at:
- https://pgmpy.org/
- https://pgmpy.org/models/bayesiannetwork.html
- https://pgmpy.org/factors/discrete.html
- https://pgmpy.org/exact_infer/ve.html

## Implementation in Python using `pgmpy`.

Let's start by installing the `pgmpy` library.

In [ ]:
!pip install -q pgmpy networkx matplotlib plotly daft

Import the required libraries.

In [ ]:
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
import networkx as nx
import matplotlib.pyplot as plt

**2. Umsetzung und Beantwortung von Fragen mittels pgmpy. Abzugeben ist ein Jupyter Notebook. Sie können sich bei Ihrer Lösung an https://github.com/jku-win-dke/SemAI/blob/main/V08_PGM.ipynb orientieren. (insgesamt 5 Punkte)**

Es sind jeweils folgende Aufgaben zu erfüllen bzw Fragen zu beantworten (jeweils 1 Punkt):




a) Erstellung des Bayes’schen Netzes (Struktur + Wahrscheinlichkeiten)

In [ ]:
# Define the network structure
model = BayesianNetwork([('TrainStrike', 'PeterLate'), ('TrainStrike', 'JohannaLate'), ('PetersAlarmFails', 'PeterLate')])

statenames = { 'TrainStrike': ['True', 'False'], 'PeterLate': ['True', 'False'], 'JohannaLate': ['True', 'False'], 'PetersAlarmFails': ['True', 'False']}

# Define the probability distribution for 'TrainStrike' (P(T))
cpd_trainstrike = TabularCPD(variable='TrainStrike', variable_card=2,
                              state_names=statenames,
                              values=[[0.05], [0.95]])

# Define the probability distribution for 'PetersAlarmFails' (P(A))
cpd_petersalarmfails = TabularCPD(variable='PetersAlarmFails', variable_card=2,
                                    state_names=statenames,
                                    values=[[0.1], [0.9]])

# Define the conditional probability distribution for 'JohannaLate' (P(J|T))
cpd_johannalate = TabularCPD(variable='JohannaLate', variable_card=2,
                              state_names=statenames,
                              evidence=['TrainStrike'], evidence_card=[2],
                              values=[[0.5, 0.02],
                                      [0.5, 0.98]])

# Define the conditional probability distribution for 'PeterLate' (P(P|T,A))
cpd_peterlate = TabularCPD(variable='PeterLate', variable_card=2,
                              state_names=statenames,
                              evidence=['PetersAlarmFails', 'TrainStrike'], evidence_card=[2, 2],
                              values=[[0.7, 0.4, 0.3, 0.01],
                                      [0.3, 0.6, 0.7, 0.99]])

# Add the probability distributions to the model
model.add_cpds(cpd_trainstrike, cpd_petersalarmfails, cpd_johannalate, cpd_peterlate)
model.to_daft(node_pos={"TrainStrike": (0, 2), "JohannaLate": (0, 0), "PeterLate": (2, 0), "PetersAlarmFails": (2, 2)}).render()

print(cpd_trainstrike)
print(cpd_petersalarmfails)
print(cpd_johannalate)
print(cpd_peterlate)


b) Wie hoch ist die Wahrscheinlichkeit, dass Johanna bzw Peter zu spät in die Arbeit kommen?

In [ ]:
from pgmpy.inference import VariableElimination
inference = VariableElimination(model)

print(inference.query(variables=['JohannaLate']))

c) Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, falls Peters Wecker nicht funktioniert?

In [ ]:
evidence={'PetersAlarmFails':'True'}
result = inference.query(variables=['JohannaLate'], evidence=evidence)
print(evidence)
print(result)

d) Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, wenn Peter zu spät in die Arbeit kommt?

In [ ]:
evidence={'PeterLate':'True'}
result = inference.query(variables=['JohannaLate'], evidence=evidence)
print(evidence)
print(result)

e) Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, wenn Peter zu spät in die Arbeit kommt obwohl Peters Wecker funktioniert?

In [ ]:
evidence={'PeterLate':'True', 'PetersAlarmFails':'False'}
result = inference.query(variables=['JohannaLate'], evidence=evidence)
print(evidence)
print(result)